In [1]:
library(data.table)
library(dplyr)
library(stringr)
library(ggplot2)


Attache Paket: ‘dplyr’


Die folgenden Objekte sind maskiert von ‘package:data.table’:

    between, first, last


Die folgenden Objekte sind maskiert von ‘package:stats’:

    filter, lag


Die folgenden Objekte sind maskiert von ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
variant <- fread("variant_with_maf_new.csv")
variant <- variant[variant$rare==TRUE,]
head(variant)

Identifier,sampleID,frequency,dbSNP id (g),Gene name (d),Entrez gene id (e),Chromosome (f),Start (f),End (f),Cytoband (f),AA Impact (h),1000 Genome (i),ESP5400 (j),SIFT Score (k),SNP Type (l),Polyphen (m),Variant class (n),AF,rare
<chr>,<chr>,<dbl>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<lgl>
chr10:100010849_G_A,ME:SK-MEL-2,45.238,-,LOXL4,84171,10,100010849,100010849,10q24,R725W,0,0.00000,0.00,Missense,1.000,Type 2,1e-06,TRUE
chr10:100013418_T_G,ME:SK-MEL-2,56.140,rs148260647,LOXL4,84171,10,100013418,100013418,10q24,Y576S,0,0.00019,0.32,Missense,1.000,Type 1,6e-05,TRUE
chr10:100017532_G_T,LC:EKVX,82.456,-,LOXL4,84171,10,100017532,100017532,10q24,R379S,0,0.00000,0.04,Missense,0.999,Type 2,9e-03,TRUE
chr10:100020867_C_A,CO:HCT-15,38.889,-,LOXL4,84171,10,100020867,100020867,10q24,E158D,0,0.00000,0.53,Missense,0.956,Type 2,1e-06,TRUE
chr10:100021911_C_T,BR:MCF7,29.508,rs139735732,LOXL4,84171,10,100021911,100021911,10q24,G113R,0,0.00019,0.58,Missense,0.000,Type 1,2e-04,TRUE
chr10:100167409_A_C,CO:HCC-2998,44.000,-,PYROXD2,84795,10,100167409,100167409,10q24.2,F82C,0,0.00000,0.00,Missense,0.999,Type 2,1e-06,TRUE


In [3]:
variant$sampleID <- variant$sampleID %>%
  strsplit( ":")  %>%
  sapply( tail, 1 ) %>%
  str_replace_all("-","")%>%
  str_replace_all(" ","")%>%
  str_replace("(TB)","")%>%
  str_replace("UO31","U031")%>%
  str_replace_all('\\(|\\)', "")%>%
  str_replace_all("A549/ATCC","A549")%>%
  str_replace_all("7860","X786O")%>%
  str_replace_all("MDAN","MDAMB468")%>%
  str_replace_all("NCI/ADRRES","NCIADRES")

In [4]:
head(variant)

Identifier,sampleID,frequency,dbSNP id (g),Gene name (d),Entrez gene id (e),Chromosome (f),Start (f),End (f),Cytoband (f),AA Impact (h),1000 Genome (i),ESP5400 (j),SIFT Score (k),SNP Type (l),Polyphen (m),Variant class (n),AF,rare
<chr>,<chr>,<dbl>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<lgl>
chr10:100010849_G_A,SKMEL2,45.238,-,LOXL4,84171,10,100010849,100010849,10q24,R725W,0,0.00000,0.00,Missense,1.000,Type 2,1e-06,TRUE
chr10:100013418_T_G,SKMEL2,56.140,rs148260647,LOXL4,84171,10,100013418,100013418,10q24,Y576S,0,0.00019,0.32,Missense,1.000,Type 1,6e-05,TRUE
chr10:100017532_G_T,EKVX,82.456,-,LOXL4,84171,10,100017532,100017532,10q24,R379S,0,0.00000,0.04,Missense,0.999,Type 2,9e-03,TRUE
chr10:100020867_C_A,HCT15,38.889,-,LOXL4,84171,10,100020867,100020867,10q24,E158D,0,0.00000,0.53,Missense,0.956,Type 2,1e-06,TRUE
chr10:100021911_C_T,MCF7,29.508,rs139735732,LOXL4,84171,10,100021911,100021911,10q24,G113R,0,0.00019,0.58,Missense,0.000,Type 1,2e-04,TRUE
chr10:100167409_A_C,HCC2998,44.000,-,PYROXD2,84795,10,100167409,100167409,10q24.2,F82C,0,0.00000,0.00,Missense,0.999,Type 2,1e-06,TRUE


In [5]:
variant <- variant[,c(1,2,5)]
colnames(variant)[3] <- "geneID"
head(variant)


Identifier,sampleID,geneID
<chr>,<chr>,<chr>
chr10:100010849_G_A,SKMEL2,LOXL4
chr10:100013418_T_G,SKMEL2,LOXL4
chr10:100017532_G_T,EKVX,LOXL4
chr10:100020867_C_A,HCT15,LOXL4
chr10:100021911_C_T,MCF7,LOXL4
chr10:100167409_A_C,HCC2998,PYROXD2


In [6]:
training <- fread("outrider_res.txt")
training <- training[,-1]
features <- unique(training$featureID)
length(features)

Warning message in fread("outrider_res.txt"):
“Detected 19 column names but the data has 20 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file.”


[1] 6975

In [7]:
prot <- fread("proteinGroups.txt")
prot <- prot[,c(1,7)]
colnames(prot)[1] <- 'featureID'
prot$geneID <- lapply(prot$`Gene names`, function(x){
    strsplit(x,";")[[1]][1]
})
prot$geneID <- as.character(prot$geneID)
prot <- prot[prot$featureID %in% features, ]
nrow(prot)
head(prot)

[1] 6975

featureID,Gene names,geneID
<chr>,<chr>,<chr>
P0DN26;A0A075B759;Q9Y536;F5H284;A0A0B4J2A2;P0DN37,PPIAL4E;PPIAL4A;PPIAL4D;PPIAL4C,PPIAL4E
A0AV96;A0AV96-2,RBM47,RBM47
A0AVF1;A0AVF1-2;A0AVF1-3,TTC26,TTC26
A0AVT1;A0AVT1-2;A0AVT1-3;A0AVT1-4,UBA6,UBA6
A0FGR8-6;A0FGR8,ESYT2,ESYT2
A0JLT2;A0JLT2-2,MED19,MED19


In [9]:
variant_prot <- merge(variant, prot[,-2], all.x=TRUE)
head(variant_prot)
nrow(variant_prot)

geneID,Identifier,sampleID,featureID
<chr>,<chr>,<chr>,<chr>
-,chr10:22607063_G_A,HOP62,NA
-,chr10:22607893_T_C,HCC2998,NA
-,chr10:22618232_G_A,HCC2998,NA
-,chr10:22618256_G_T,NCIH322M,NA
-,chr11:119004793_G_T,HCT15,NA
-,chr11:18254071_T_A,SF268,NA


[1] 69706

In [10]:
variant_prot[is.na(variant_prot$featureID),]

geneID,Identifier,sampleID,featureID
<chr>,<chr>,<chr>,<chr>
-,chr10:22607063_G_A,HOP62,NA
-,chr10:22607893_T_C,HCC2998,NA
-,chr10:22618232_G_A,HCC2998,NA
-,chr10:22618256_G_T,NCIH322M,NA
-,chr11:119004793_G_T,HCT15,NA
-,chr11:18254071_T_A,SF268,NA
-,chr11:193112_C_T,MCF7,NA
-,chr11:193112_C_T,MDAMB231,NA
-,chr11:193112_C_T,HS578T,NA


In [11]:
variant_prot[!is.na(variant_prot$featureID),]

geneID,Identifier,sampleID,featureID
<chr>,<chr>,<chr>,<chr>
A2ML1,chr12:8976409_C_A,HCC2998,A8K2U0;A8K2U0-2
A2ML1,chr12:8988109_A_C,HCC2998,A8K2U0;A8K2U0-2
A2ML1,chr12:8990937_C_A,LOXIMVI,A8K2U0;A8K2U0-2
A2ML1,chr12:8991708_G_T,EKVX,A8K2U0;A8K2U0-2
A2ML1,chr12:9000230_C_T,HCC2998,A8K2U0;A8K2U0-2
A2ML1,chr12:9001419_T_C,RPMI8226,A8K2U0;A8K2U0-2
A2ML1,chr12:9006840_G_A,SNB75,A8K2U0;A8K2U0-2
A2ML1,chr12:9013789_A_C,KM12,A8K2U0;A8K2U0-2
A2ML1,chr12:9016563_*_-GC,SF295,A8K2U0;A8K2U0-2


In [ ]:
prot_samp_withRare <- variant_prot[,-c(1:2)]
prot_samp_withRare <- prot_samp_withRare[!duplicated(prot_samp_withRare),]
prot_samp_withRare$rare <- 1
head(prot_samp_withRare)
nrow(prot_samp_withRare)

In [ ]:
training$sampleID <- gsub('_NCI60','',training$sampleID)
training <- training[!is.na(training$pValue),] # only underexpressed
training <- training[training$log2fc<0,] # only underexpressed
nrow(training)
head(training)

res1 <- merge(training[,c(1,2,3)], prot_samp_withRare, by = c('sampleID', 'featureID'), all=TRUE)
head(res1[is.na(res1$pValue),])

In [ ]:
res <- merge(training[,c(1,2,3)], prot_samp_withRare, by = c('sampleID', 'featureID'), all.x=TRUE)
res[is.na(res$rare)]$rare <- 0
res <- res[order(pValue),]
head(res)
nrow(res)

In [ ]:
res$cumsum <- cumsum(res$rare)
res$rank = 1:nrow(res)
res$proportion <- res$cumsum/res$rank
head(res)

In [ ]:
ggplot(res, aes(x = log(rank), y = proportion)) + geom_line() + geom_point()

In [ ]:
missing_pred <- fread('missing_pred.csv')
missing_pred$sampleID <- gsub("_NCI60","",missing_pred$sampleID)
head(missing_pred)
nrow(missing_pred)

res2 <- merge(missing_pred, prot_samp_withRare, by = c('sampleID', 'featureID'), all=TRUE)
head(res2[is.na(res2$missing),])

In [ ]:
res_impute <- merge(missing_pred, prot_samp_withRare, by = c('sampleID', 'featureID'), all.x=TRUE)
res_impute[is.na(res_impute$rare)]$rare <- 0
res_impute <- res_impute[order(predprob, -rare),]
head(res_impute)
nrow(res_impute)

In [ ]:
res_impute$cumsum <- cumsum(res_impute$rare)
res_impute$rank = 1:nrow(res_impute)
res_impute$proportion <- res_impute$cumsum/res_impute$rank
head(res_impute)

In [ ]:
max(res_impute$cumsum)

In [ ]:
max(res$cumsum)

In [ ]:
ggplot(res_impute, aes(log(rank), y = proportion)) + geom_line() + geom_point()

In [ ]:
colnames(res)[7] <- "Baseline"
colnames(res_impute)[10] <- "withImputedMissingValue"

In [ ]:
result <- merge(res[,6:7], res_impute[,9:10], by = "rank")
head(result)

In [ ]:
ggplot(result, aes(x = log2(rank))) + geom_line(aes(y = Baseline, colour = "Baseline"), size = 1.2) +
    geom_line(aes(y = withImputedMissingValue, colour = "withImputedMissingValue"), size = 1.2) #+
    #xlim(100,40000)